**Abandonned for the moment**

In [3]:
import numpy as np
import pandas as pd

In [6]:
pol_wu_df = pd.read_csv('../analyze/pollen_weather.csv')
pol_wu_df.set_index('Date')
pol_wu_df.drop(columns=['Year','Month','DayOfYear','Ambrosia','Artemisia','Asteraceae','Betula','Ericaceae',
                             'Carpinus','Castanea','Quercus','Chenopodium','Cupressaceae','Acer','Fraxinus',
                             'Gramineae','Fagus','Juncaceae','Aesculus','Larix','Corylus','Juglans',
                             'Umbellifereae','Ulmus','Urtica','Rumex','Populus','Pinaceae','Plantago','Platanus',
                             'Salix','Cyperaceae','Filipendula','Sambucus','Tilia','Events'], inplace=True)
pol_wu_df.head(5)

,Date,Alnus,TempMax,TempAvg,TempMin,DewPointMax,DewPointAvg,DewPointMin,HumidMax,HumidAvg,HumidMin,SeaLevelPressureMax,SeaLevelPressureAvg,SeaLevelPressureMin,VisibilityMax,VisibilityAvg,VisibilityMin,WindMax,WindAvg,Precip
0,1996-07-01,0.0,17.0,13.0,10.0,10.0,8.0,5.0,94.0,74.0,45.0,1011.0,1009.0,1008.0,10.0,9.0,4.0,35.0,19.0,0.0
1,1996-07-02,0.0,14.0,11.0,8.0,10.0,8.0,7.0,94.0,80.0,67.0,1013.0,1011.0,1008.0,10.0,10.0,7.0,23.0,13.0,0.0
2,1996-07-03,0.0,22.0,16.0,10.0,12.0,8.0,5.0,94.0,63.0,38.0,1014.0,1012.0,1010.0,10.0,10.0,5.0,32.0,11.0,0.0
3,1996-07-04,0.0,19.0,17.0,14.0,14.0,13.0,11.0,94.0,82.0,60.0,1013.0,1010.0,1008.0,10.0,8.0,3.0,39.0,23.0,0.0
4,1996-07-05,0.0,17.0,14.0,12.0,15.0,13.0,10.0,94.0,89.0,82.0,1013.0,1009.0,1005.0,10.0,9.0,5.0,24.0,13.0,0.0
